In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import statsmodels.api as sm
from scipy import stats
import pwlf
from sklearn import mixture
import scipy.linalg as la
from statsmodels.graphics.gofplots import qqplot_2samples
from scipy import stats
from scipy.interpolate import CubicSpline
from localreg import *
import os.path
from os import path
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from matplotlib import gridspec

import rpy2
# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
# R package names
packnames = ('lokern')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))


from rpy2.robjects import FloatVector

lokern = rpackages.importr('lokern')


from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, HPacker, VPacker


(as ‘lib’ is unspecified)



In [2]:
%matplotlib qt

The following cell loads all raw cleaned data from the COPAS BIOSORT and imaging platform. Medians for each well are calculated to determine the conversion from BIOSORT units to microns. 

In [3]:
data_all = []
with open('../data/raw_cleaned.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        data_all.append(row)

headerS = list(np.array(data_all[0])[[0,1,12,14,15,20,17,18,19]])

dataS = np.array(data_all)[1:,[0,1,12,14,15,20,17,18,19]]
for i in range(len(dataS)):
    dataS[i,1] = dataS[i,1][-1]

dataS = dataS.astype(float)

data_allI = []
with open('../data/all_student_measurements.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        data_allI.append(row)
        
header_allI = list(np.array(data_allI[0]))

student = np.array(data_allI[1:])[:,5]

headerI = ['Hour', 'replicate', 'well', 'length', 'width']

names = np.unique(student)

dataI = np.array(data_allI[1:])[:,[2,4,4,7,8]]

for i in range(len(dataI)):
    dataI[i,1] = ord(dataI[i,1][0])-64
    dataI[i,2] = dataI[i,2][1:]

dataI = dataI.astype(float)

header_combined_all = ['replicate','hour','TOF','length','norm.EXT','width','EXT','length x width']
medians_combined_all = np.zeros((6*72,8))
count = 0
for t in range(72):
    SortTime = dataS[dataS[:,0]==t+1]
    ImTime = dataI[dataI[:,0]==t+1]
    for r in range(6):
        SortRep = SortTime[SortTime[:,1]==r+1]
        ImRep = ImTime[ImTime[:,1]==r+1]
        timeofflight = SortRep[:,3]
        extinction = SortRep[:,5]
        length = ImRep[:,3]
        width = ImRep[:,4]
        ext = SortRep[:,4]
        area = np.multiply(length, width)
        medians_combined_all[count] = np.array([r,t,np.median(timeofflight),np.median(length),
                                               np.median(extinction),np.median(width),
                                                np.median(ext), np.median(area)])
        count+=1
medians_combined_all=medians_combined_all[medians_combined_all[:,0]!=0]

TOF_length = np.poly1d(np.polyfit(medians_combined_all[:,2],medians_combined_all[:,3], 1))
nEXT_width = np.poly1d(np.polyfit(medians_combined_all[:,4],medians_combined_all[:,5], 1))
def VolumeS(tof,normEXT):
    return np.multiply(TOF_length(tof),np.square(np.divide(nEXT_width(normEXT),2.)))*np.pi
def VolumeI(length, width):
    return np.multiply(length,np.square(np.divide(width,2.)))*np.pi

The following cell performs the bootstrap regression on red fluorescence, sorter calculated length, and sorter calculated width. This cell may be skipped if you wish to use our previously calculated bootstrap results

In [11]:
#Bootstrap to all points
iterations = 2000
fit_R_boot = np.empty((6,iterations,1000))
fit_L_boot = np.empty((6,iterations,1000))
fit_W_boot = np.empty((6,iterations,1000))
regtime = np.linspace(1,73,1000)
Combo = np.divide(dataS[:,-1],VolumeS(dataS[:,3],dataS[:,5])*1e-6)
Volume = VolumeS(dataS[:,3],dataS[:,5])*1e-6
for r in range(6):
        Rep = r+1
        boot_obs = dataS[dataS[:,1]==Rep,-1]
        boot_obsL = TOF_length(dataS[dataS[:,1]==Rep,3])
        boot_obsW = nEXT_width(dataS[dataS[:,1]==Rep,5])
        boot_time = dataS[dataS[:,1]==Rep,0]
        for i in range(iterations):
            number_samples = len(boot_obs)
            count = np.arange(len(boot_obs))
            pick = np.random.choice(count,len(count),replace = True)
            boot_data = boot_obs[pick] 
            boot_x = boot_time[pick] 
            boot_dataL = boot_obsL[pick] 
            boot_dataW = boot_obsW[pick] 
            fit_R_boot[r,i] = lokern.lokerns(FloatVector(boot_x),FloatVector(boot_data),0, 1000,
                                            FloatVector(regtime),False, 2, True)[3]
            fit_L_boot[r,i] = lokern.lokerns(FloatVector(boot_x),FloatVector(boot_dataL),0, 1000,
                                            FloatVector(regtime),False, 2, True)[3]
            fit_W_boot[r,i] = lokern.lokerns(FloatVector(boot_x),FloatVector(boot_dataW),0, 1000,
                                            FloatVector(regtime),False, 2, True)[3]
        print('Done with Replicate {}'.format(r+1))
        
np.save('../data/Red_time_fit.npy', fit_R_boot)
np.save('../data/Length_time_fit.npy',fit_L_boot)
np.save('../data/Width_time_fit.npy',fit_W_boot)
np.save('../data/time_fit.npy', regtime)

Done with Replicate 1
Done with Replicate 2
Done with Replicate 3
Done with Replicate 4
Done with Replicate 5
Done with Replicate 6


Following cell loads previously calculated bootstrap results for the regressions of red fluorescence, length, and width.

In [4]:
fit_R_boot = np.load('../data/Red_time_fit.npy')
fit_L_boot = np.load('../data/Length_time_fit.npy')
fit_W_boot = np.load('../data/Width_time_fit.npy')
regtime = np.load('../data/time_fit.npy')
Volume = VolumeS(dataS[:,3],dataS[:,5])*1e-6
iterations = 2000
Combo = np.divide(dataS[:,-1],VolumeS(dataS[:,3],dataS[:,5])*1e-6)
datalength = TOF_length(dataS[:,3])
datawidth = nEXT_width(dataS[:,5])
datarep = dataS[:,1]

The following cell generates the plots in figure 7

In [10]:
ExRep = 2
maxtime = 50
import seaborn as sns

growing = np.empty((np.shape(regtime[regtime<=maxtime])))
for i in range(len(growing)):
    if np.logical_and(regtime[i]>=1, regtime[i]<=13):
        growing[i] = True
    elif np.logical_and(regtime[i]>=17, regtime[i]<=22):
        growing[i] = True
    elif np.logical_and(regtime[i]>=27, regtime[i]<=32):
        growing[i] = True
    elif np.logical_and(regtime[i]>=39, regtime[i]<=45):
        growing[i] = True
    elif np.logical_and(regtime[i]>=50, regtime[i]<=56):
        growing[i] = True
    else:
        growing[i] = False

################
Rep = ExRep-1

d = 10



ax = plt.subplot(1,2,1)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
V = ((0.25*np.pi)*np.multiply(fit_L_boot[Rep,:,regtime<=maxtime],
                              np.square(fit_W_boot[Rep,:,regtime<=maxtime])))*1e-6

scalevalue = np.max(np.mean((np.divide(fit_R_boot[Rep,:,regtime<=maxtime],V)),axis = 1))

plt.fill_between(regtime[regtime<=maxtime],\
                 np.min((np.divide(fit_R_boot[Rep,:,regtime<=maxtime],V))/scalevalue,axis = 1),\
                 np.max((np.divide(fit_R_boot[Rep,:,regtime<=maxtime],V))/scalevalue,axis = 1),\
                 color = 'steelblue',alpha = 0.4)

plt.plot(regtime[regtime<=maxtime], np.mean((np.divide(fit_R_boot[Rep,:,regtime<=maxtime],V)),axis = 1)/scalevalue,
         label = 'Regression', color = 'steelblue')

sns.regplot(dataS[dataS[:,1] == Rep+1,0], (Combo[dataS[:,1] == Rep+1])/scalevalue,fit_reg = False,\
            x_jitter = 0.1,color = 'black', label = 'Data Points',scatter_kws = {'alpha' : 0.4,'s':1})


plt.ylabel('Pumping Frequency*Corpus Fraction', fontsize = 15)
plt.xlim([1,maxtime])
plt.ylim([0,2])
plt.xlabel('Time (Hours)',fontsize = 15)
###################################################################3
V = ((0.25*np.pi)*np.multiply(fit_L_boot[Rep,:,regtime<=maxtime],
                               np.square(fit_W_boot[Rep,:,regtime<=maxtime])))*1e-6
dV = np.gradient(V,regtime[regtime<=maxtime], axis = 0)
dV2 = np.gradient(dV,regtime[regtime<=maxtime], axis = 0)
dR = np.gradient(fit_R_boot[Rep,:,regtime<=maxtime],regtime[regtime<=maxtime], axis = 0)
dR_mean = np.mean(dR, axis = 1)
dV_mean = np.median(dV,axis = 1)
R_mean = np.mean(fit_R_boot[Rep,:,regtime<=maxtime],axis = 1)

dV2_ave = np.empty((len(dV_mean)-2*d,iterations))
dR_ave = np.empty((len(dR_mean))-2*d)
for i in range(len(dV_mean)-2*d):
    dV2_ave[i,:] = np.mean(dV2[i:i+2*d],axis = 0)
    dR_ave[i] = np.mean(dR_mean[i:i+2*d])

ax2 = plt.subplot(2,2,2)

ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.xaxis.set_ticks_position('bottom')
ax2.yaxis.set_ticks_position('left')
dV2_mean = np.mean(dV2, axis = 1)
dR_mean = np.mean(dR, axis = 1)
notgrowth = (np.logical_and(np.sign(np.mean(dV2_ave[:],axis = 1))>0,np.sign(dR_ave)>0))
notshrink = (np.logical_and(np.sign(np.mean(dV2_ave[:],axis = 1))<0,np.sign(dR_ave)<0))

plt.fill_between(regtime[regtime<=maxtime][d:-d],np.zeros(len(regtime[regtime<=maxtime][d:-d])),
                 np.max(R_mean)*1.1*np.ones(len(regtime[regtime<=maxtime][d:-d])),
                 where = notgrowth, color = 'darkseagreen', alpha = 0.2)

plt.fill_between(regtime[regtime<=maxtime][d:-d],np.zeros(len(regtime[regtime<=maxtime][d:-d])),
                 np.max(R_mean)*1.1*np.ones(len(regtime[regtime<=maxtime][d:-d])),
                 where = notshrink, color = 'rosybrown', alpha = 0.2)

plt.fill_between(regtime[regtime<=maxtime], np.min((dV),axis = 1),\
                 np.max((dV),axis = 1),\
                 color = 'steelblue',alpha = 0.4)
plt.plot(regtime[regtime<=maxtime], ((dV_mean)), label = '$\\frac{dV}{dt}$',\
         color = 'steelblue')

plt.plot(regtime[regtime<=maxtime], ((dV_mean)), label = '$\\frac{dV}{dt}$',\
         color = 'steelblue')

plt.ylabel('Growth Rate', fontsize = 15)
plt.xlim([1,maxtime])
plt.ylim([0,np.max(dV_mean)*1.1])

#################################
ax3 = plt.subplot(2,2,4)
ax3.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)
ax3.xaxis.set_ticks_position('bottom')
ax3.yaxis.set_ticks_position('left')


plt.fill_between(regtime[regtime<=maxtime][d:-d],np.zeros(len(regtime[regtime<=maxtime][d:-d])),
                 np.max(R_mean)*1.1*np.ones(len(regtime[regtime<=maxtime][d:-d])),
                 where = notgrowth, color = 'darkseagreen', alpha = 0.2)

plt.fill_between(regtime[regtime<=maxtime][d:-d],np.zeros(len(regtime[regtime<=maxtime][d:-d])),
                 np.max(R_mean)*1.1*np.ones(len(regtime[regtime<=maxtime][d:-d])),
                 where = notshrink, color = 'rosybrown', alpha = 0.2)
plt.fill_between(regtime[regtime<=maxtime],
                 np.min((fit_R_boot[Rep,:,regtime<=maxtime]),axis = 1),
                 np.max((fit_R_boot[Rep,:,regtime<=maxtime]),axis = 1),
                 color = 'steelblue',alpha = 0.4)

plt.plot(regtime[regtime<=maxtime], (R_mean), label = 'Red',color = 'steelblue')


plt.ylabel('Red Fluorescence', fontsize = 15)
plt.xlim([1,maxtime])
plt.ylim([0,np.max(R_mean)*1.1])
plt.xlabel('Time (Hours)', fontsize = 15)




plt.show()

/usr/local/lib/python3.9/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


The following cell lists the larval steady growth hours as determined by using normalized red inflection points

In [6]:
L1 = [[1,13],[1,13],[1,13],[1,13],[1,13],[1,13]]
L2 = [[16.52,21.93],[16.37,22.39],[16.72,21.98],[16.89,23.22],[17.38,22.36],[17.19,23.40]]
L3 = [[26.75,31.94],[26.93,32.96],[26.94,32.89],[27.23,33.10],[27.58,33.63],[27.72,33.86]]
L4 = [[38.20,44.41],[38.37,43.50],[38.66,45.31],[38.56,43.49],[38.99,45.07],[38.96,44.48]]

stages = [L1,L2,L3,L4]

The following cell calculates the bootstrap regressions for length and width in each larval stage individually. This cell may be skipped if you wish to use previously calculated booststrap results

In [ ]:
fit_Ls_boot = np.empty((6,4,iterations,100))
fit_Ws_boot = np.empty((6,4,iterations,100))
reglength = np.empty((6,4, 100))
regtime_s = np.empty((6,4,100))
medrep = medians_combined[:,0]
medhours = medians_combined[:,1]
medlength = TOF_length(medians_combined[:,3])
medwidth = nEXT_width(medians_combined[:,5])

datarep = dataS[:,1]
datalength = TOF_length(dataS[:,3])
datawidth = nEXT_width(dataS[:,5])
datatime = dataS[:,0]

for r in range(6):
    Rep = r+1
    for s in range(4):
        stage = stages[s][r]
        hourmin = np.ceil(stage[0])
        hourmax = np.floor(stage[1])

        bootlength_all = datalength[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin,datatime<=hourmax))]
        bootwidth_all = datawidth[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin,datatime<=hourmax))]
        boottime = datatime[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin,datatime<=hourmax))]
        regtime_s[r,s] = np.linspace(hourmin,hourmax,100)
        numbersamples = len(bootlength_all)


        for i in range(iterations):
            count = np.arange(numbersamples)
            pick = np.random.choice(count,numbersamples,replace = True)
            boot_length = bootlength_all[pick]
            boot_width = bootwidth_all[pick]
            boot_time = boottime[pick]
            fit_Ls_boot[r,s,i] = lokern.lokerns(FloatVector(boot_time),FloatVector(boot_length),0, 100,
                                                FloatVector(regtime_s[r,s]),False, 2, True)[3]
            fit_Ws_boot[r,s,i] = lokern.lokerns(FloatVector(boot_time),FloatVector(boot_width),0, 100,
                                                FloatVector(regtime_s[r,s]),False, 2, True)[3]
    print('Finished with Replicate {}'.format(r+1))
    

np.save('../data/Length_byStage.npy',fit_Ls_boot)
np.save('../data/Width_byStage.npy',fit_Ws_boot)
np.save('../data/LW_regtime_bystage.npy',regtime_s)

This cell loads previously calculated bootstrapped regressions for length and width at each larval stage individually

In [7]:
iterations = 2000
fit_Ls_boot = np.load('../data/Length_byStage.npy')
fit_Ws_boot = np.load('../data/Width_byStage.npy')
regtime_s = np.load('../data/LW_regtime_bystage.npy')

The following cell calculates instantaneous values of $\frac{\Delta W}{\Delta L}$ and the errors associated with this fraction from the bootstrap regressions

In [8]:
fit_dLs_boot = np.empty((6,4,iterations,100))
fit_dWs_boot = np.empty((6,4,iterations,100))
mean_dW = np.empty((6,4,100))
mean_dL = np.empty((6,4,100))

for r in range(6):
    for s in range(4):
        fit_dLs_boot[r,s] = np.gradient(fit_Ls_boot[r,s],regtime_s[r,s],axis = 1)
        fit_dWs_boot[r,s] = np.gradient(fit_Ws_boot[r,s],regtime_s[r,s],axis = 1)
        mean_dL[r,s] = np.gradient(np.mean(fit_Ls_boot[r,s],axis = 0),regtime_s[r,s])
        mean_dW[r,s] = np.gradient(np.mean(fit_Ws_boot[r,s],axis = 0),regtime_s[r,s])
        
LWcov = np.empty((2,2,6,4,100))

for r in range(6):
    for s in range(4):
        for t in range(100):
            l = fit_dLs_boot[r,s,:,t]
            w = fit_dWs_boot[r,s,:,t]
            lw = np.array([l,w])
            LWcov[:,:,r,s,t] = np.cov(lw)
            
error = np.sqrt(np.divide(LWcov[1,1],np.square(mean_dL))+
                np.divide(np.square(mean_dW)*LWcov[0,0],np.power(mean_dL,4))-
                np.divide(2*mean_dW*LWcov[0,1],np.power(mean_dL,3)))

The following cell produces the plots in figure 8

In [9]:
c_cycle = ['#4477AA', '#117733', '#DDCC77', '#CC6677']
for r in range(1,2):
    plt.figure(figsize=(13,8))
    Rep = r+1
    gs = gridspec.GridSpec(2,1, height_ratios=[3,1]) 
    stages = [L1,L2,L3,L4]
    plt.subplot(gs[1])
    for s in range(3):
        plt.plot(regtime_s[r,s], np.divide(mean_dW[r,s],mean_dL[r,s]), color = c_cycle[s])
        plt.fill_between(regtime_s[r,s],np.divide(mean_dW[r,s],mean_dL[r,s])-error[r,s],
                         np.divide(mean_dW[r,s],mean_dL[r,s])+error[r,s], color = c_cycle[s],alpha = 0.4)
    #plt.xlim([270,650])
    #plt.xticks([])
    plt.ylabel('Slope: $\\frac{dW}{dL}$', fontsize = 15)
    #plt.ylim([-0.05,0.5])
    plt.xlabel('Time (Hours)', fontsize = 15)

    plt.subplot(gs[0])
    plt.hist2d(datalength[datarep==Rep],datawidth[datarep==Rep],bins = 200, cmap = 'gray')
    plt.colorbar()

    for s in range(4):
        plt.plot(np.mean(fit_Ls_boot[r,s],axis = 0),
                 np.mean(fit_Ws_boot[r,s],axis = 0)
                 ,color = c_cycle[s],lw = 2)
        counts,xbins,ybins = np.histogram2d((fit_Ls_boot[r,s]).flatten(),(fit_Ws_boot[r,s]).flatten(),
                                            bins=100,normed=True)
        plt.contourf(counts.transpose(),extent=[xbins[0],xbins[-1],ybins[0],ybins[-1]],
                     colors = c_cycle[s],
                     alpha = 0.4, levels = [0.1*np.max(counts),np.max(counts)])
        plt.contour(counts.transpose(),extent=[xbins[0],xbins[-1],ybins[0],ybins[-1]],
                     colors = c_cycle[s],
                     linewidths = 0.5, levels = [0.1*np.max(counts)])
    plt.ylim([15,50])
    plt.xlim([270,650])
    plt.ylabel('Width ($\mu$m)', fontsize = 15)
    plt.xlabel('Length ($\mu$m)', fontsize = 15)
    plt.suptitle('Fit to Replicate {}'.format(Rep), fontsize = 20)
    plt.show()

The following cell calculates the defecation analysis from the supplement and produces figure S5

In [13]:
Td = 45*(1./3600.)
g = 0.43

Rep = 2
red = np.mean(fit_R_boot[Rep-1], axis = 0)
drdt = np.mean(np.gradient(fit_R_boot[Rep-1].T, regtime,axis = 0), axis = 1)
red_std = np.std(fit_R_boot[Rep-1],axis = 0)
dFdt = np.mean(np.gradient(fit_R_boot[Rep-1].T, regtime, axis = 0) + fit_R_boot[Rep-1].T*g/Td, axis = 1)
dFdt_std = np.std(np.gradient(fit_R_boot[Rep-1].T, regtime, axis = 0) + fit_R_boot[Rep-1].T*g/Td, axis = 1)
plt.plot(regtime, dFdt/np.max(dFdt), color = 'steelblue', lw =2, ls = 'solid',
         label = '$\\frac{d Red}{dt} + \\frac{Red \cdot h_d}{T_d}$')
plt.plot(regtime, (red)/np.max(red),
         color = 'palevioletred', label = 'Red Fluorescence',lw = 2, ls = 'dashed')
plt.fill_between(regtime, (dFdt+dFdt_std)/np.max(dFdt),(dFdt-dFdt_std)/np.max(dFdt),
                 color = 'steelblue', alpha = 0.5, lw = 2)
plt.fill_between(regtime, (red+red_std)/np.max(red),(red-red_std)/np.max(red),
                 color = 'palevioletred', alpha = 0.5, ls = 'dashed', lw = 2)
plt.legend(loc = 'best', fontsize = 13)
plt.ylim([0,1.1])
plt.xlim([regtime.min(),regtime.max()])
plt.xlabel('Time (Hours)', fontsize = 15)
plt.ylabel('Value Scaled by Maximum', fontsize = 15)
plt.show()